# Creating Radar Charts

In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import pathlib
import warnings
from scipy import stats
from mplsoccer import PyPizza, FontManager

pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_json(r'C:\Users\elvin\Documents\Soccermatics\Wyscout\events\events_Italy.json')

#remove any rows where we do not have the start and end of an action available
mask = train['positions'].apply(lambda x: len(x)==2)
train = train[mask]

## Calculating xG value

In [ ]:
def calculatexG(df, npxG):
    """
    Parameters
    ----------
    df : dataframe
        dataframe with Wyscout event data.
    npxG : boolean
        True if xG should not include penalties, False elsewhere.

    Returns
    -------
    xG_sum: dataframe
        dataframe with sum of Exppected Goals for players during the season.

    """
    #creating a basic xG model
    shots = df[df['eventName']=="Shot"].copy()
    shots["X"] = shots.positions.apply(lambda cell: (100 - cell[0]['x']) * 105/100)
    shots["Y"] = shots.positions.apply(lambda cell: cell[0]['y'] * 68/100)
    shots["C"] = shots.positions.apply(lambda cell: abs(cell[0]['y'] - 50) * 68/100)
    shots["Distance"] = np.sqrt(shots["X"]**2 + shots["C"]**2)
    shots["Angle"] = np.where(np.arctan(7.32 * shots["X"] / (shots["X"]**2 + shots["C"]**2 - (7.32/2)**2)) > 0, 
                              np.arctan(7.32 * shots["X"] /(shots["X"]**2 + shots["C"]**2 - (7.32/2)**2)), 
                              np.arctan(7.32 * shots["X"] /(shots["X"]**2 + shots["C"]**2 - (7.32/2)**2)) + np.pi)
    shots['Goal'] = shots.tags.apply(lambda x: 1 if {'id':101}in x else 0).astype(object)
    #the header id tag is 403
    headers = shots.loc[shots.apply(lambda x:{'id':403} in x.tags)]
    non_headers = shots.drop(headers.index)

    headers_model = smf.glm(formula="Goal ~ Distance + Angle", data=headers,
                            family=sm.families.Binomial()).fit()
    
    non_headers_model = smf.glm(formula="Goal ~ Distance + Angle", data=non_headers,
                            family=sm.families.Binomial()).fit()
    

    

647372